In [1]:
# PLEASE MODIFY ABSOLUTE PATH AT 4TH BLOCK

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
from torchvision import transforms, models
from PIL import Image
import torch.nn as nn
import os
import csv

In [4]:
# PLEASE MODIFY ABSOLUTE PATH HERE
dic_path = '/content/drive/MyDrive/大學/3/機學/testfinal'

In [5]:
# assign .csv address
csvPath = f'{dic_path}/110550130_submission.csv'
resCsv = open(csvPath, 'w')

# load the test data directory
test_data_dir = f'{dic_path}/test'

# Load the trained model file
model_path = f'{dic_path}/110550130_model.pt'

numClass = 200

In [6]:
def load_and_preprocess_image(image_path):
    # Load and preprocess the image for prediction
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    input_image = transform(image)
    input_image = input_image.unsqueeze(0)  # Add batch dimension
    return input_image


In [7]:
def predict_image(model, image_path, device):
    # Load and preprocess the image
    input_image = load_and_preprocess_image(image_path)
    input_image = input_image.to(device)

    model.eval()

    with torch.no_grad():
        outputs = model(input_image)
        _, predicted_class = torch.max(outputs, 1)

    return predicted_class.item()

In [8]:
def set_labels():
  # labels = {}

  # for dir_name in os.listdir(train_data_dir):
  #   key = int(dir_name[0:3])-1
  #   labels[key] = dir_name

  labels = {0: '001.Black_footed_Albatross', 1: '002.Laysan_Albatross', 2: '003.Sooty_Albatross', 3: '004.Groove_billed_Ani', 4: '005.Crested_Auklet', 5: '006.Least_Auklet', 6: '007.Parakeet_Auklet', 8: '009.Brewer_Blackbird', 14: '015.Lazuli_Bunting', 12: '013.Bobolink', 15: '016.Painted_Bunting', 7: '008.Rhinoceros_Auklet', 13: '014.Indigo_Bunting', 9: '010.Red_winged_Blackbird', 10: '011.Rusty_Blackbird', 11: '012.Yellow_headed_Blackbird', 16: '017.Cardinal', 17: '018.Spotted_Catbird', 18: '019.Gray_Catbird', 25: '026.Bronzed_Cowbird', 24: '025.Pelagic_Cormorant', 26: '027.Shiny_Cowbird', 27: '028.Brown_Creeper', 20: '021.Eastern_Towhee', 21: '022.Chuck_will_Widow', 22: '023.Brandt_Cormorant', 19: '020.Yellow_breasted_Chat', 23: '024.Red_faced_Cormorant', 28: '029.American_Crow', 29: '030.Fish_Crow', 38: '039.Least_Flycatcher', 34: '035.Purple_Finch', 35: '036.Northern_Flicker', 32: '033.Yellow_billed_Cuckoo', 36: '037.Acadian_Flycatcher', 31: '032.Mangrove_Cuckoo', 30: '031.Black_billed_Cuckoo', 33: '034.Gray_crowned_Rosy_Finch', 37: '038.Great_Crested_Flycatcher', 39: '040.Olive_sided_Flycatcher', 40: '041.Scissor_tailed_Flycatcher', 41: '042.Vermilion_Flycatcher', 51: '052.Pied_billed_Grebe', 42: '043.Yellow_bellied_Flycatcher', 50: '051.Horned_Grebe', 43: '044.Frigatebird', 45: '046.Gadwall', 44: '045.Northern_Fulmar', 49: '050.Eared_Grebe', 48: '049.Boat_tailed_Grackle', 47: '048.European_Goldfinch', 46: '047.American_Goldfinch', 52: '053.Western_Grebe', 53: '054.Blue_Grosbeak', 56: '057.Rose_breasted_Grosbeak', 60: '061.Heermann_Gull', 57: '058.Pigeon_Guillemot', 55: '056.Pine_Grosbeak', 54: '055.Evening_Grosbeak', 58: '059.California_Gull', 62: '063.Ivory_Gull', 59: '060.Glaucous_winged_Gull', 61: '062.Herring_Gull', 63: '064.Ring_billed_Gull', 64: '065.Slaty_backed_Gull', 65: '066.Western_Gull', 67: '068.Ruby_throated_Hummingbird', 69: '070.Green_Violetear', 71: '072.Pomarine_Jaeger', 68: '069.Rufous_Hummingbird', 66: '067.Anna_Hummingbird', 72: '073.Blue_Jay', 73: '074.Florida_Jay', 70: '071.Long_tailed_Jaeger', 74: '075.Green_Jay', 75: '076.Dark_eyed_Junco', 76: '077.Tropical_Kingbird', 82: '083.White_breasted_Kingfisher', 81: '082.Ringed_Kingfisher', 85: '086.Pacific_Loon', 83: '084.Red_legged_Kittiwake', 79: '080.Green_Kingfisher', 80: '081.Pied_Kingfisher', 77: '078.Gray_Kingbird', 78: '079.Belted_Kingfisher', 84: '085.Horned_Lark', 86: '087.Mallard', 87: '088.Western_Meadowlark', 88: '089.Hooded_Merganser', 92: '093.Clark_Nutcracker', 97: '098.Scott_Oriole', 89: '090.Red_breasted_Merganser', 90: '091.Mockingbird', 91: '092.Nighthawk', 95: '096.Hooded_Oriole', 96: '097.Orchard_Oriole', 94: '095.Baltimore_Oriole', 93: '094.White_breasted_Nuthatch', 98: '099.Ovenbird', 99: '100.Brown_Pelican', 100: '101.White_Pelican', 104: '105.Whip_poor_Will', 103: '104.American_Pipit', 102: '103.Sayornis', 101: '102.Western_Wood_Pewee', 106: '107.Common_Raven', 109: '110.Geococcyx', 108: '109.American_Redstart', 107: '108.White_necked_Raven', 105: '106.Horned_Puffin', 110: '111.Loggerhead_Shrike', 117: '118.House_Sparrow', 116: '117.Clay_colored_Sparrow', 114: '115.Brewer_Sparrow', 119: '120.Fox_Sparrow', 118: '119.Field_Sparrow', 115: '116.Chipping_Sparrow', 113: '114.Black_throated_Sparrow', 112: '113.Baird_Sparrow', 111: '112.Great_Grey_Shrike', 120: '121.Grasshopper_Sparrow', 121: '122.Harris_Sparrow', 124: '125.Lincoln_Sparrow', 122: '123.Henslow_Sparrow', 130: '131.Vesper_Sparrow', 125: '126.Nelson_Sharp_tailed_Sparrow', 126: '127.Savannah_Sparrow', 127: '128.Seaside_Sparrow', 123: '124.Le_Conte_Sparrow', 129: '130.Tree_Sparrow', 128: '129.Song_Sparrow', 131: '132.White_crowned_Sparrow', 132: '133.White_throated_Sparrow', 137: '138.Tree_Swallow', 135: '136.Barn_Swallow', 140: '141.Artic_Tern', 134: '135.Bank_Swallow', 138: '139.Scarlet_Tanager', 139: '140.Summer_Tanager', 141: '142.Black_Tern', 136: '137.Cliff_Swallow', 133: '134.Cape_Glossy_Starling', 142: '143.Caspian_Tern', 143: '144.Common_Tern', 144: '145.Elegant_Tern', 145: '146.Forsters_Tern', 148: '149.Brown_Thrasher', 146: '147.Least_Tern', 153: '154.Red_eyed_Vireo', 152: '153.Philadelphia_Vireo', 150: '151.Black_capped_Vireo', 149: '150.Sage_Thrasher', 151: '152.Blue_headed_Vireo', 147: '148.Green_tailed_Towhee', 154: '155.Warbling_Vireo', 155: '156.White_eyed_Vireo', 156: '157.Yellow_throated_Vireo', 157: '158.Bay_breasted_Warbler', 165: '166.Golden_winged_Warbler', 166: '167.Hooded_Warbler', 161: '162.Canada_Warbler', 162: '163.Cape_May_Warbler', 164: '165.Chestnut_sided_Warbler', 159: '160.Black_throated_Blue_Warbler', 160: '161.Blue_winged_Warbler', 158: '159.Black_and_white_Warbler', 163: '164.Cerulean_Warbler', 167: '168.Kentucky_Warbler', 168: '169.Magnolia_Warbler', 169: '170.Mourning_Warbler', 170: '171.Myrtle_Warbler', 179: '180.Wilson_Warbler', 175: '176.Prairie_Warbler', 172: '173.Orange_crowned_Warbler', 178: '179.Tennessee_Warbler', 174: '175.Pine_Warbler', 171: '172.Nashville_Warbler', 177: '178.Swainson_Warbler', 173: '174.Palm_Warbler', 176: '177.Prothonotary_Warbler', 180: '181.Worm_eating_Warbler', 181: '182.Yellow_Warbler', 182: '183.Northern_Waterthrush', 183: '184.Louisiana_Waterthrush', 192: '193.Bewick_Wren', 185: '186.Cedar_Waxwing', 188: '189.Red_bellied_Woodpecker', 191: '192.Downy_Woodpecker', 186: '187.American_Three_toed_Woodpecker', 184: '185.Bohemian_Waxwing', 190: '191.Red_headed_Woodpecker', 189: '190.Red_cockaded_Woodpecker', 187: '188.Pileated_Woodpecker', 193: '194.Cactus_Wren', 194: '195.Carolina_Wren', 195: '196.House_Wren', 196: '197.Marsh_Wren', 198: '199.Winter_Wren', 199: '200.Common_Yellowthroat', 197: '198.Rock_Wren'}


  return labels

In [9]:
# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
writer = csv.writer(resCsv)
writer.writerow(['id','label'])

model = models.resnet50()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, numClass)
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()

labels = set_labels()
print(labels)

for img_name in os.listdir(test_data_dir):
  img_path = os.path.join(test_data_dir, img_name)
  predicted_class = predict_image(model, img_path, device)

  predicted_label = labels[predicted_class]

  writer.writerow([img_name[:-4],predicted_label])
  print(f'Index:{predicted_class} class:{predicted_label}')


{0: '001.Black_footed_Albatross', 1: '002.Laysan_Albatross', 2: '003.Sooty_Albatross', 3: '004.Groove_billed_Ani', 4: '005.Crested_Auklet', 5: '006.Least_Auklet', 6: '007.Parakeet_Auklet', 8: '009.Brewer_Blackbird', 14: '015.Lazuli_Bunting', 12: '013.Bobolink', 15: '016.Painted_Bunting', 7: '008.Rhinoceros_Auklet', 13: '014.Indigo_Bunting', 9: '010.Red_winged_Blackbird', 10: '011.Rusty_Blackbird', 11: '012.Yellow_headed_Blackbird', 16: '017.Cardinal', 17: '018.Spotted_Catbird', 18: '019.Gray_Catbird', 25: '026.Bronzed_Cowbird', 24: '025.Pelagic_Cormorant', 26: '027.Shiny_Cowbird', 27: '028.Brown_Creeper', 20: '021.Eastern_Towhee', 21: '022.Chuck_will_Widow', 22: '023.Brandt_Cormorant', 19: '020.Yellow_breasted_Chat', 23: '024.Red_faced_Cormorant', 28: '029.American_Crow', 29: '030.Fish_Crow', 38: '039.Least_Flycatcher', 34: '035.Purple_Finch', 35: '036.Northern_Flicker', 32: '033.Yellow_billed_Cuckoo', 36: '037.Acadian_Flycatcher', 31: '032.Mangrove_Cuckoo', 30: '031.Black_billed_Cucko

In [11]:
resCsv.close()

In [12]:
!python --version

Python 3.10.12
